In [150]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import key_dict
gkey=key_dict["maps"]

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [151]:
cities_data=pd.read_csv("output_data/cities.csv")
cities_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hanstholm,57.1167,8.6167,36.00,3,0,8.21,DK,1615934081
1,Bambous Virieux,-20.3428,57.7575,78.80,78,75,8.05,MU,1615934082
2,Nueva Guinea,11.6876,-84.4562,78.28,65,84,9.75,NI,1615934082
3,Boa Vista,2.8197,-60.6733,82.40,69,40,9.22,BR,1615934244
4,Virginia Beach,36.8529,-75.9780,46.40,87,90,8.05,US,1615934281


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [152]:
gmaps.configure(api_key=gkey)
fig=gmaps.figure()
locations=cities_data[["Lat","Lng"]].astype(float)
Humidity=cities_data["Humidity"].astype(float)
heat_layer=gmaps.heatmap_layer(locations,weights=Humidity,
                               dissipating=False,max_intensity=Humidity.max(),
                               point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [153]:
vacation_cities=cities_data.loc[
                                (cities_data["Max Temp"].astype(float)>70)
                                &(cities_data["Max Temp"].astype(float)<80)
                                &(cities_data["Wind Speed"].astype(float)<10)
                                &(cities_data["Cloudiness"].astype(float)<5)]
vacation_cities






,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,Allāpalli,19.4167,80.0667,72.39,38,0,4.65,IN,1615934154
106,Adrar,20.5022,-10.0711,72.82,14,0,9.37,MR,1615934212
267,Atmakūr,15.8833,78.5833,72.66,48,0,2.71,IN,1615934675
349,Robertson,36.5167,-86.8667,71.01,43,1,4.61,US,1615934758
412,Dutlwe,-23.9833,23.9000,70.59,53,2,5.23,BW,1615934853
450,Salalah,17.0151,54.0924,75.20,64,0,3.44,OM,1615934905
499,Ipixuna,-1.7625,-48.8043,78.80,89,0,8.05,BR,1615934936
510,Lamu,-2.2717,40.9020,78.80,89,1,6.91,KE,1615934972
534,Coruripe,-10.1256,-36.1756,77.43,74,4,8.52,BR,1615934987


In [154]:
vacation_cities.isnull().sum()

City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [155]:
Hotel_df =vacation_cities[["City","Lat","Lng","Country"]]
Hotel_df["Hotel Name"]=" "
Hotel_df


/opt/anaconda3/envs/pythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Country,Hotel Name
66,Allāpalli,19.4167,80.0667,IN,
106,Adrar,20.5022,-10.0711,MR,
267,Atmakūr,15.8833,78.5833,IN,
349,Robertson,36.5167,-86.8667,US,
412,Dutlwe,-23.9833,23.9000,BW,
450,Salalah,17.0151,54.0924,OM,
499,Ipixuna,-1.7625,-48.8043,BR,
510,Lamu,-2.2717,40.9020,KE,
534,Coruripe,-10.1256,-36.1756,BR,


In [156]:

base_url=base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params={"radius":5000,"type":"lodging","key":gkey}
for index,row in Hotel_df.iterrows():
    params["location"]=f'{row["Lat"]},{row["Lng"]}'
    response=requests.get(base_url,params=params).json()
    results=response["results"]
    try:
        Hotel_df.loc[index,"Hotel Name"]=results[0]["name"]
#         pprint(results[0]["name"])
    except:
        Hotel_df.loc[index,"Hotel Name"]=None
        #print(f'Can not find a hotel near the city: {row["City"]}')
Hotel_df

/opt/anaconda3/envs/pythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Country,Hotel Name
66,Allāpalli,19.4167,80.0667,IN,Hotel IK Residency
106,Adrar,20.5022,-10.0711,MR,None
267,Atmakūr,15.8833,78.5833,IN,Mee seva atk
349,Robertson,36.5167,-86.8667,US,Hampton Inn Springfield
412,Dutlwe,-23.9833,23.9000,BW,None
450,Salalah,17.0151,54.0924,OM,HAMDAN PLAZA HOTEL SALALAH
499,Ipixuna,-1.7625,-48.8043,BR,Casa da Oscarina
510,Lamu,-2.2717,40.9020,KE,Lamu House
534,Coruripe,-10.1256,-36.1756,BR,Pousada e Hotel Litoral Sul


In [157]:
Hotel_df.isnull().sum()

City          0
Lat           0
Lng           0
Country       0
Hotel Name    2
dtype: int64

In [158]:
Hotel_df=Hotel_df.dropna(axis=0)
Hotel_df

,City,Lat,Lng,Country,Hotel Name
66,Allāpalli,19.4167,80.0667,IN,Hotel IK Residency
267,Atmakūr,15.8833,78.5833,IN,Mee seva atk
349,Robertson,36.5167,-86.8667,US,Hampton Inn Springfield
450,Salalah,17.0151,54.0924,OM,HAMDAN PLAZA HOTEL SALALAH
499,Ipixuna,-1.7625,-48.8043,BR,Casa da Oscarina
510,Lamu,-2.2717,40.9020,KE,Lamu House
534,Coruripe,-10.1256,-36.1756,BR,Pousada e Hotel Litoral Sul


In [167]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
locations=Hotel_df[["Lat","Lng"]]
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
# Add marker layer ontop of heat map
# gmaps.configure(api_key=gkey)
# fig=gmaps.figure()
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))